In [1]:
#import modin.pandas as pd
from pandas_profiling import ProfileReport
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import (GridSearchCV, StratifiedKFold,
                                     train_test_split)
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, mean_absolute_error,
                             roc_auc_score)
from sklearn.impute import SimpleImputer
from sklearn.ensemble import (AdaBoostClassifier, RandomForestClassifier,
                              RandomForestRegressor)
import xgboost as xgb
import scipy.stats as stats
import requests
import pyforest
import cv2 as cv
import bs4 as bs
import warnings
import time
import re
import random
import os
import math
import datetime
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
# from autoviz.AutoViz_Class import AutoViz_Class

%config InlineBackend.figure_format = 'retina'

#import keras as ks
# import tensorflow as tf

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
sns.set(style='whitegrid', color_codes=True, font_scale=1.5,
        palette='plasma', font='Arial', rc={'figure.figsize': (8, 6)})


# Predict survival on the titanic dataset

## - With one hot encoding


In [2]:
df=sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True


In [3]:
df.drop(['age','fare'],axis=1,inplace=True)

In [4]:
df.head()

,survived,pclass,sex,sibsp,parch,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,1,0,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,1,0,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,0,0,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,1,0,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,0,0,S,Third,man,True,NaN,Southampton,no,True


In [5]:
df = pd.get_dummies(df,drop_first=True) # pd.get_dummies(df,drop_first=True) creates dummy variables for categorical variables in the dataframe df and drops the first dummy variable for each categorical variable. 
# Useful for OneHotEncoding

In [6]:
df.head()

,survived,pclass,sibsp,parch,adult_male,alone,sex_male,embarked_Q,embarked_S,class_Second,class_Third,who_man,who_woman,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,embark_town_Queenstown,embark_town_Southampton,alive_yes
0,0,3,1,0,True,False,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0
1,1,1,1,0,False,False,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1
2,1,3,0,0,False,True,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,1
3,1,1,1,0,False,False,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,1
4,0,3,0,0,True,True,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0


In [7]:
df.isna().sum()

survived                   0
pclass                     0
sibsp                      0
parch                      0
adult_male                 0
alone                      0
sex_male                   0
embarked_Q                 0
embarked_S                 0
class_Second               0
class_Third                0
who_man                    0
who_woman                  0
deck_B                     0
deck_C                     0
deck_D                     0
deck_E                     0
deck_F                     0
deck_G                     0
embark_town_Queenstown     0
embark_town_Southampton    0
alive_yes                  0
dtype: int64

In [8]:
X = df.drop('survived', axis=1)
y = df['survived']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train.shape
X_test.shape

(712, 21)

(179, 21)

In [11]:
# Binary classification
# KNN
import sklearn.neighbors as knn
knn_model = knn.KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
# acc=knn_model.score(X_test, y_test)
pred = knn_model.predict(X_test)
pred


KNeighborsClassifier()

array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1], dtype=int64)

In [12]:
print(f"Accuracy on training set: {knn_model.score(X_train, y_train):.3f}")
print(f"Accuracy on test set: {knn_model.score(X_test, y_test):.3f}")

Accuracy on training set: 0.968
Accuracy on test set: 0.955


## - Without one hot encoding

In [13]:
df=sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True


In [14]:
df=df.select_dtypes(exclude=['object'])

In [15]:
df.dropna(inplace=True)

In [16]:
df.head()

,survived,pclass,age,sibsp,parch,fare,class,adult_male,deck,alone
1,1,1,38.00,1,0,71.28,First,False,C,False
3,1,1,35.00,1,0,53.10,First,False,C,False
6,0,1,54.00,0,0,51.86,First,True,E,True
10,1,3,4.00,1,1,16.70,Third,False,G,False
11,1,1,58.00,0,0,26.55,First,False,C,True


In [17]:
# Binary classification
# KNN
import sklearn.neighbors as knn
knn_model = knn.KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
# acc=knn_model.score(X_test, y_test)
pred = knn_model.predict(X_test)
pred


KNeighborsClassifier()

array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1], dtype=int64)

In [18]:
print(f"Accuracy on training set: {knn_model.score(X_train, y_train):.3f}")
print(f"Accuracy on test set: {knn_model.score(X_test, y_test):.3f}")

Accuracy on training set: 0.968
Accuracy on test set: 0.955
